## Timetabling Program for St. George's School
#### Copyright  © 2020 Hoshino Math Services

In [1]:
"""This script consists of a heuristic named 'Progressive Assignment', which
generates a close-to-optimal timetable for grade 11/12 Students at St.George's School 
for the academic year 2020-2021, assuming a post-COVID world with cohort partitioning."""

# Import Python Modules

#import sys
#!{sys.executable} -m pip install ortools 

import time
import numpy as np
import pandas as pd
from random import random
from random import shuffle
from ortools.linear_solver import pywraplp

### The Data

In [2]:
# Import Student and Course Data 

InputFile = pd.ExcelFile("Input Data.xlsx") 
StudentMatrix = pd.read_excel(InputFile, 'Student Selections') 
StudentInfo = StudentMatrix.values.tolist()
CourseMatrix = pd.read_excel(InputFile, 'Sections') 
CourseInfo = CourseMatrix.values.tolist()


In [3]:
# Generate the list of courses, list of students, and class lists
# NOTE: Career Life Connections = 99, Spare 12 = 231, Spare 11 = 232
    
CourseIDList = [99, 231, 232]
for i in range(len(CourseInfo)):
    CourseID = CourseInfo[i][1]
    if not CourseID in CourseIDList:
        CourseIDList.append(CourseID)
CourseIDList.sort()

StudentIDList = []
for i in range(len(StudentInfo)):
    StudentID = StudentInfo[i][0]
    if not StudentID in StudentIDList:
        StudentIDList.append(StudentID)
StudentIDList.sort()

n = len(StudentIDList)
m = len(CourseIDList)

StudentChoices = [ [] for x in range(n)]
for i in range(len(StudentInfo)):
    StudentID = StudentInfo[i][0]
    StudentIndex = StudentIDList.index(StudentID)
    CourseName = StudentInfo[i][4]
    if CourseName in CourseIDList:
        CourseIndex = CourseIDList.index(CourseName)
        if not CourseIndex in StudentChoices[StudentIndex]:
            StudentChoices[StudentIndex].append(CourseIndex)

            
# Generate the list of grade 11, grade 12, and boarding students          
Grade11Students = []
Grade12Students = []
BoardingStudents = []

for i in range(len(StudentInfo)):
    StudentID = StudentInfo[i][0]
    StudentIndex = StudentIDList.index(StudentID)    
    if StudentInfo[i][1] == 'Grade 11':
        Grade11Students.append(StudentIndex) 
    if StudentInfo[i][1] == 'Grade 12':
        Grade12Students.append(StudentIndex) 
    if StudentInfo[i][2] == 'Boarding':
        BoardingStudents.append(StudentIndex)

Grade11Students = list(set(Grade11Students))
Grade11Students.sort()  
Grade12Students = list(set(Grade12Students))
Grade12Students.sort()  
BoardingStudents = list(set(BoardingStudents))
BoardingStudents.sort()   

CourseNameList = ['' for i in range(m)]
for i in range(len(CourseInfo)):
    CourseName = CourseInfo[i][0]
    CourseID = CourseInfo[i][1]
    CourseIndex = CourseIDList.index(CourseID)
    CourseNameList[CourseIndex] = CourseName
    
ClassList = [ [] for x in range(m)]
Enrollment = [ 0 for x in range(m)]
for i in range(n):
    for j in range(len(StudentChoices[i])):
        CourseIndex = StudentChoices[i][j]
        Enrollment[CourseIndex] += 1
        if not i in ClassList[CourseIndex]:
            ClassList[CourseIndex].append(i)  

            
# Generate lists of courses divided by priority (Must, High, Medium, Low)            
MustPriority = []
HighPriority = []
MedPriority = []
TripletList = [ [] for t in range(len(CourseInfo))]
TripletCap = [ 0 for t in range(len(CourseInfo))]
CombinedLG = []

indexcount = 0

for i in range(len(CourseInfo)):

    CourseID = CourseInfo[i][1]
    CourseIndex = CourseIDList.index(CourseID)    
    Section = CourseInfo[i][4]
    Block = CourseInfo[i][5]
    Capacity = CourseInfo[i][6]
    TripletList[indexcount] = [CourseIndex, Section, Block]
    
    TripletCap[indexcount] = Capacity
    if CourseInfo[i][11] == 'n' or CourseInfo[i][11] == 'N':
        TripletCap[indexcount] = 0    
        
    if CourseInfo[i][10] == 'y' or CourseInfo[i][10] == 'Y':
        CombinedLG.append(indexcount)
 
    indexcount += 1
        
    if CourseInfo[i][7] == 'Y': 
        if not CourseIndex in HighPriority:
            if not CourseInfo[i][13] == 'Y' and not CourseInfo[i][13] == 'y':
                HighPriority.append(CourseIndex)
    if CourseInfo[i][8] == 'Y':
        if not CourseIndex in MedPriority:
            MedPriority.append(CourseIndex)        
    if CourseInfo[i][13] == 'Y' or CourseInfo[i][13] == 'y':
        if not CourseIndex in MustPriority:
            MustPriority.append(CourseIndex) 
        

In [4]:
# Find all the Triplets that belong to the same course, and to the same block

Blocks = ['A','B','C','D','E','F','G','H']

TripletIDs = [ [] for r in range(m)]
TripletBlocks = [ [] for r in range(8)]

for t in range(len(TripletList)):
    ID = TripletList[t][0]
    Block = TripletList[t][2]
    TripletIDs[ID].append(t)
    for r in range(8):
        if Blocks[r]==Block:
            TripletBlocks[r].append(t)

In [5]:
# Manually list out all the pairs of rows (t) that must be assigned to the same Learning Group

SameLGList = [[38,41], [38,43], [41,43], [39,42], [39,44], [42,44], [40,45], [46,48], [47,49],
              [166,168], [167,169], [170,172], [171,173], [154,155], [194,195]]

print("These course sections are to be combined into the same unique Learning Group")
print("")
for t in SameLGList:
    if not t[0] in CombinedLG:
        print(TripletList[t[0]][1], t, CourseNameList[TripletList[t[0]][0]])
        print(TripletList[t[1]][1], t, CourseNameList[TripletList[t[1]][0]])
        print("")
    

These course sections are to be combined into the same unique Learning Group

2H [154, 155] Spanish 11
1H [154, 155] Spanish 12

1E [194, 195] Yearbook Media Design 11
1E [194, 195] Yearbook Media Design 12



In [6]:
# Generate the set of courses requested by Grade 11/12 students
# We do not include Spare or Career-Life or any classes with zero enrollment
# or any courses cancelled by Sarah, the Associate Principle of Academics at SGS

ActualCourses = []
for j in range(m):
    if Enrollment[j]>0 and len(TripletIDs[j]) > 0:
        ActualCourses.append(j)

LowPriority = list(set(ActualCourses)-set(HighPriority)-set(MedPriority)-set(MustPriority))        
        
AllSingles = []
AllMultiples = []

for j in ActualCourses:
    if len(TripletIDs[j]) == 1: AllSingles.append(j)
    if len(TripletIDs[j]) > 1: AllMultiples.append(j)

print("There are", n, "Grade 11/12 students requesting a total of",
     len(ActualCourses), "courses")
print("Of these", len(ActualCourses), "courses,", len(AllSingles), "of them are singletons and",
     len(AllMultiples), "have multiple sections")
print("We are not scheduling Career Life Connections")

There are 328 Grade 11/12 students requesting a total of 89 courses
Of these 89 courses, 41 of them are singletons and 48 have multiple sections
We are not scheduling Career Life Connections


In [7]:
print("All Grade 11 students must have one of these English courses")
for j in [54, 89, 55, 90]:
    print(CourseIDList[j], CourseNameList[j])
    
print("")

print("All Grade 12 students must have one of these English courses")
for j in [46, 49, 56]:
    print(CourseIDList[j], CourseNameList[j])

All Grade 11 students must have one of these English courses
368 Composition 11
492 Composition 11 Honours
370 Literary Studies 11
493 Literary Studies 11 (Honours)

All Grade 12 students must have one of these English courses
337 English 12 AP (Language)
353 English 12 AP (Literature)
371 English Studies 12


In [8]:
# In the timetable we sent to SGS, we got 807 out of 808 must courses satisfied.
# The one exception was student i=243 who requested j=90.  We give that student j=55
# instead, replacing Literary Studies 11 (Honours) with Literary Studies 11

StudentChoices[243].remove(90)
StudentChoices[243].append(55)

### The Preference Matrix

In [9]:
# Set preference coefficients
# Assume weight of 5-3-1 for high/medium/low courses

P = np.zeros((n,m), dtype=int)

for i in range(n):
    for j in StudentChoices[i]:
        if not j in [15,16]:
            P[i,j] = 1
            if j in MustPriority: P[i,j] = 10
            if j in HighPriority: P[i,j] = 5 
            if j in MedPriority: P[i,j] = 3
        
P.shape

(328, 103)

In [10]:
# Count the theoretical maximum happiness score for priority courses
# Note that the Low count is off (789 vs. 1165) due to the Spare 11 and Spare 12 course, and the
# Career Life course that takes place outside of school hours.

MustOccurrences = sum(np.count_nonzero(P == 10, axis=1))
MustMaxScore = 10*sum(np.count_nonzero(P == 10, axis=1))
HighOccurrences = sum(np.count_nonzero(P == 5, axis=1))
HighMaxScore = 5*sum(np.count_nonzero(P == 5, axis=1))
MedOccurrences = sum(np.count_nonzero(P == 3, axis=1))
MedMaxScore = 3*sum(np.count_nonzero(P == 3, axis=1))
LowOccurrences = sum(np.count_nonzero(P == 1, axis=1))
LowMaxScore = 1*sum(np.count_nonzero(P == 1, axis=1))
AllAssignments = MustOccurrences+HighOccurrences+MedOccurrences+LowOccurrences
TheoreticalMax = MustMaxScore+HighMaxScore+MedMaxScore+LowMaxScore
MaximumScore = sum(sum(P[i][j] for j in range(m)) for i in range(n))


print('Total course requests:', AllAssignments, ' w/ theoretical maximum happiness score:',  TheoreticalMax)
print('Total Must Priority course requests:', MustOccurrences, ' w/ happiness score:',  MustMaxScore)
print('Total High Priority course requests:', HighOccurrences, ' w/ happiness score:',  HighMaxScore)
print('Total Medium Priority course requests:', MedOccurrences, ' w/ happiness score:',  MedMaxScore)
print('Total Low Priority course requests:', LowOccurrences, ' w/ happiness score:',  LowMaxScore)
print(sum(np.count_nonzero(P != 0, axis = 1))==AllAssignments, TheoreticalMax==MaximumScore)

Total course requests: 2466  w/ theoretical maximum happiness score: 12225
Total Must Priority course requests: 807  w/ happiness score: 8070
Total High Priority course requests: 541  w/ happiness score: 2705
Total Medium Priority course requests: 166  w/ happiness score: 498
Total Low Priority course requests: 952  w/ happiness score: 952
True True


## PART 1: timetabling pre-Covid
No need to partition students into learning groups

### The ILP

In [11]:
# Assignment of students to fixed blocks, without any regard to Learning Groups
# In other words, pretend we have 1 learning group with all 328 students

solver = pywraplp.Solver('SGS', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

start_time = time.time()

Students = range(n)
Courses = ActualCourses
Triplets = range(len(TripletList))
LearningGroups = [1]
Blocks = ['A','B','C','D','E','F','G','H']

# define boolean variables
x = {}
for t in Triplets:
    for l in LearningGroups:
        x[t,l] = solver.IntVar(0,1, 'x[%d,%d]' % (t,l))

y = {}
for i in Students:
    for l in LearningGroups:
        y[i,l] = solver.IntVar(0,1, 'y[%d,%d]' % (i,l))
                           
z = {}
for i in Students:
    for t in Triplets:
        for l in LearningGroups:
            z[i,t,l] = solver.IntVar(0,1, 'z[%d,%d,%d]' % (i,t,l)) 

            
# CONSTRAINT 1: Ensure each triplet is assigned to at most ONE learning group, unless otherwise specified
for t in Triplets:
    if t in CombinedLG:
        solver.Add(sum(x[t,l] for l in LearningGroups) <= 4)
    else:
        solver.Add(sum(x[t,l] for l in LearningGroups) <= 1)

        
# CONSTRAINT 2: Ensure each student is assigned to at most ONE learning group
for i in Students:
    solver.Add(sum(y[i,l] for l in LearningGroups) <= 1)
              
    
# CONSTRAINT 3: No student can take a course unless it is available to students in their learning group
for i in Students:
    for t in Triplets:
        for l in LearningGroups:
            solver.Add(z[i,t,l] <= y[i,l])
            solver.Add(z[i,t,l] <= x[t,l])
                
                                
# CONSTRAINT 4: No student can take more than one course in a block
for i in Students:
    for r in range(8):
        solver.Add(sum(z[i,t,l] for l in LearningGroups for t in TripletBlocks[r]) <= 1)
              
        
# CONSTRAINT 5: NO student can take the same course twice       
for i in Students:
    for j in Courses:
        solver.Add(sum(z[i,t,l] for l in LearningGroups for t in TripletIDs[j]) <= 1)


# CONSTRAINT 6: Do not assign course j to a student i if P[i,j]=0      
for i in Students:
    for j in Courses:
        if P[i,j]==0:
            solver.Add(sum(z[i,t,l] for l in LearningGroups for t in TripletIDs[j]) == 0)
         
                    
# CONSTRAINT 7: No course section can exceed its prescribed capacity
for t in Triplets:
    cap = TripletCap[t]
    solver.Add(sum(z[i,t,l] for l in LearningGroups for i in Students) <= cap)
    
    
# CONSTRAINT 8: For the 807 must courses, the student must be assigned to a section of
# that course, though the block/section is NOT fixed.
for i in Students:
    for j in Courses:
        if P[i,j]==10:
            solver.Add(sum(z[i,t,l] for l in LearningGroups for t in TripletIDs[j]) == 1)
  
        
solver.Maximize(solver.Sum(P[i,TripletList[t][0]]*z[i,t,l] 
                           for i in Students for t in Triplets for l in LearningGroups))
               
sol = solver.Solve()

print("")
print("PART 1 - Optimization without any consideration of Learning Groups")
print('Optimization Complete with Total Happiness Score of', round(solver.Objective().Value()))

# compute runtime
solving_time = time.time() - start_time

print('The code ran in', round(solving_time,1), 'seconds')


PART 1 - Optimization without any consideration of Learning Groups
Optimization Complete with Total Happiness Score of 11927
The code ran in 7.3 seconds


### Statistics

In [12]:
# Descriptive Statistics of our results

MustCount=0
HighCount=0
MedCount=0
LowCount=0
MustTotal=0
HighTotal=0
MedTotal=0
LowTotal=0

for i in Students:
    for j in Courses:
        if P[i,j]>0 and not j in [15,16]:
            if j in MustPriority: MustTotal +=1
            elif j in HighPriority: HighTotal +=1
            elif j in MedPriority: MedTotal +=1
            else: LowTotal +=1
        
for i in Students:
    for t in Triplets:
        for l in LearningGroups:
            if z[i,t,l].solution_value() == 1:
                j = TripletList[t][0]
                if not j in [15,16]:
                    if j in MustPriority: MustCount +=1
                    elif j in HighPriority: HighCount +=1
                    elif j in MedPriority: MedCount +=1
                    else: LowCount +=1
                    
Total = MustTotal+HighTotal+MedTotal+LowTotal
Count = MustCount+HighCount+MedCount+LowCount

print(MustCount, "out of", MustTotal, "mandatory priority course assignments were made:", 
     round(100*MustCount/MustTotal, 2), "percent")
print(HighCount, "out of", HighTotal, "high priority course assignments were made:", 
     round(100*HighCount/HighTotal, 2), "percent")
print(MedCount, "out of", MedTotal, "medium priority course assignments were made:", 
     round(100*MedCount/MedTotal, 2), "percent")
print(LowCount, "out of", LowTotal, "low priority course assignments were made:", 
     round(100*LowCount/LowTotal, 2), "percent")
print("Overall percentage of desired courses assigned to students:", round(100*Count/Total, 2), "percent")

807 out of 807 mandatory priority course assignments were made: 100.0 percent
533 out of 541 high priority course assignments were made: 98.52 percent
155 out of 166 medium priority course assignments were made: 93.37 percent
727 out of 789 low priority course assignments were made: 92.14 percent
Overall percentage of desired courses assigned to students: 96.48 percent


## PART 2: timetabling post-Covid
Do partition students into learning groups

In [13]:
# Create lists of Course Indices.
# Each list contains the Course Indeces of a different priority class.

Triplets = range(len(TripletList))
MustPriorityTriplets = []
HighPriorityTriplets = []
MedPriorityTriplets = []
LowPriorityTriplets = []

for t in Triplets:
    if TripletList[t][0] in MustPriority:
        MustPriorityTriplets.append(t)
    elif TripletList[t][0] in HighPriority:
        HighPriorityTriplets.append(t)
    elif TripletList[t][0] in MedPriority:
        MedPriorityTriplets.append(t)
    else:
        LowPriorityTriplets.append(t)
        
grandTotal = len(MustPriorityTriplets)+len(HighPriorityTriplets)+len(MedPriorityTriplets)+len(LowPriorityTriplets)
print(len(MustPriorityTriplets), '+', len(HighPriorityTriplets), '+', 
      len(MedPriorityTriplets), '+', len(LowPriorityTriplets), '=', len(Triplets), '=', grandTotal)
len(Triplets)==grandTotal

print('There are', len(MustPriorityTriplets), 'Must Priority Triplets')
print('There are', len(HighPriorityTriplets), 'High Priority Triplets')
print('There are', len(MedPriorityTriplets), 'Medium Priority Triplets')
print('There are', len(LowPriorityTriplets), 'Low Priority Triplets')
print('There are', len(Triplets), 'Triplets in total')

47 + 32 + 19 + 98 = 196 = 196
There are 47 Must Priority Triplets
There are 32 High Priority Triplets
There are 19 Medium Priority Triplets
There are 98 Low Priority Triplets
There are 196 Triplets in total


In [14]:
# Generate lists of Course Indeces (categorized by priority class)
# while partitioning the course indeces into the block the course ought to be offered in.

MustTripletBlocks = [ [] for r in range(8)]
MustHighTripletBlocks = [ [] for r in range(8)]
MustMedTripletBlocks = [ [] for r in range(8)]
MustLowTripletBlocks = [ [] for r in range(8)]
MHMTripletBlocks =  [ [] for r in range(8)]

for r in range(8):
    for b in TripletBlocks[r]:
        if b in MustPriorityTriplets:
            MustTripletBlocks[r].append(b)
        if b in MustPriorityTriplets or b in HighPriorityTriplets:
            MustHighTripletBlocks[r].append(b)
        if b in MustPriorityTriplets or b in HighPriorityTriplets or b in MedPriorityTriplets:
            MHMTripletBlocks[r].append(b)
        if b in MustPriorityTriplets or b in MedPriorityTriplets:
            MustMedTripletBlocks[r].append(b)
        if b in MustPriorityTriplets or b in LowPriorityTriplets:
            MustLowTripletBlocks[r].append(b)

In [15]:
# Input a suboptimal XSet for Must/High/Medium, with [807, 514, 135]

XSet = [[31, 1], [32, 2], [33, 1], [34, 3], [35, 2], [36, 2], [37, 1], [64, 3], [65, 1], [66, 1], [67, 2], [68, 3], [69, 1], [70, 3], [71, 3], [72, 2], [73, 3], [74, 1], [79, 2], [80, 3], [81, 2], [82, 1], [83, 3], [84, 2], [85, 1], [86, 2], [89, 3], [90, 2], [91, 2], [113, 1], [114, 2], [115, 3], [116, 2], [118, 1], [118, 2], [118, 3], [137, 2], [138, 1], [139, 3], [140, 1], [141, 3], [142, 2], [143, 2], [144, 2], [145, 1], [146, 2], [147, 3], [153, 2], [154, 1], [5, 3], [6, 2], [7, 1], [13, 3], [14, 1], [15, 3], [16, 3], [17, 2], [18, 1], [19, 3], [20, 1], [21, 2], [22, 2], [23, 2], [24, 2], [25, 1], [26, 3], [27, 1], [29, 1], [55, 3], [56, 1], [112, 2], [128, 3], [129, 2], [130, 1], [131, 2], [132, 2], [133, 3], [134, 1], [136, 1], [10, 1], [10, 2], [12, 1], [46, 2], [46, 3], [47, 1], [47, 2], [47, 3], [48, 1], [48, 2], [48, 3], [49, 1], [49, 2], [49, 3], [53, 1], [53, 2], [53, 3], [54, 1], [54, 2], [54, 3], [178, 2], [179, 1], [180, 3], [185, 2], [185, 3], [186, 1], [186, 2], [186, 3], [187, 1], [187, 2], [187, 3], [188, 1], [188, 2], [188, 3], [190, 3], [191, 1], [191, 3]]
YSet = [[0, 3], [1, 3], [2, 3], [3, 1], [4, 3], [5, 3], [6, 1], [7, 1], [8, 3], [9, 3], [10, 3], [11, 1], [12, 3], [13, 1], [14, 1], [15, 3], [16, 1], [17, 3], [18, 1], [19, 1], [20, 3], [21, 3], [22, 3], [23, 3], [24, 3], [25, 3], [26, 2], [27, 1], [28, 2], [29, 2], [30, 3], [31, 2], [32, 1], [33, 3], [34, 3], [35, 1], [36, 2], [37, 3], [38, 2], [39, 1], [40, 1], [41, 3], [42, 3], [43, 3], [44, 2], [45, 3], [46, 3], [47, 3], [48, 3], [49, 2], [50, 2], [51, 1], [52, 2], [53, 2], [54, 1], [55, 2], [56, 3], [57, 1], [58, 3], [59, 2], [60, 2], [61, 2], [62, 2], [63, 2], [64, 2], [65, 2], [66, 1], [67, 1], [68, 1], [69, 3], [70, 3], [71, 2], [72, 3], [73, 1], [74, 3], [75, 1], [76, 1], [77, 3], [78, 3], [79, 3], [80, 1], [81, 3], [82, 1], [83, 1], [84, 1], [85, 3], [86, 1], [87, 3], [88, 1], [89, 2], [90, 3], [91, 3], [92, 3], [93, 3], [94, 3], [95, 3], [96, 3], [97, 3], [98, 1], [99, 1], [100, 1], [101, 3], [102, 3], [103, 3], [104, 3], [105, 3], [106, 1], [107, 2], [108, 2], [109, 3], [110, 3], [111, 2], [112, 1], [113, 2], [114, 2], [115, 3], [116, 3], [117, 2], [118, 1], [119, 1], [120, 1], [121, 1], [122, 1], [123, 1], [124, 2], [125, 1], [126, 3], [127, 3], [128, 1], [129, 3], [130, 1], [131, 2], [132, 1], [133, 2], [134, 1], [135, 2], [136, 3], [137, 2], [138, 3], [139, 3], [140, 2], [141, 2], [142, 2], [143, 2], [144, 2], [145, 2], [146, 3], [147, 3], [148, 2], [149, 3], [150, 3], [151, 3], [152, 1], [153, 3], [154, 1], [155, 3], [156, 1], [157, 3], [158, 1], [159, 1], [160, 2], [161, 3], [162, 1], [163, 1], [164, 1], [165, 2], [166, 2], [167, 3], [168, 2], [169, 3], [170, 1], [171, 2], [172, 3], [173, 1], [174, 1], [175, 3], [176, 1], [177, 2], [178, 1], [179, 2], [180, 1], [181, 3], [182, 1], [183, 3], [184, 3], [185, 1], [186, 1], [187, 1], [188, 3], [189, 1], [190, 1], [191, 1], [192, 2], [193, 1], [194, 2], [195, 3], [196, 2], [197, 3], [198, 1], [199, 3], [200, 1], [201, 1], [202, 1], [203, 2], [204, 2], [205, 3], [206, 2], [207, 2], [208, 2], [209, 1], [210, 1], [211, 3], [212, 3], [213, 1], [214, 3], [215, 3], [216, 2], [217, 3], [218, 2], [219, 2], [220, 1], [221, 1], [222, 2], [223, 3], [224, 2], [225, 3], [226, 3], [227, 2], [228, 2], [229, 2], [230, 2], [231, 2], [232, 2], [233, 1], [234, 3], [235, 1], [236, 1], [237, 2], [238, 1], [239, 1], [240, 3], [241, 2], [242, 1], [243, 1], [244, 1], [245, 2], [246, 1], [247, 1], [248, 3], [249, 3], [250, 3], [251, 3], [252, 1], [253, 1], [254, 2], [255, 1], [256, 3], [257, 1], [258, 1], [259, 1], [260, 2], [261, 2], [262, 1], [263, 2], [264, 2], [265, 2], [266, 1], [267, 1], [268, 1], [269, 2], [270, 2], [271, 2], [272, 2], [273, 2], [274, 1], [275, 2], [276, 2], [277, 3], [278, 2], [279, 2], [280, 2], [281, 2], [282, 3], [283, 1], [284, 3], [285, 2], [286, 1], [287, 1], [288, 1], [289, 3], [290, 2], [291, 2], [292, 2], [293, 2], [294, 2], [295, 2], [296, 1], [297, 2], [298, 2], [299, 2], [300, 1], [301, 1], [302, 1], [303, 2], [304, 3], [305, 1], [306, 2], [307, 3], [308, 3], [309, 3], [310, 2], [311, 3], [312, 1], [313, 1], [314, 1], [315, 1], [316, 2], [317, 2], [318, 2], [319, 2], [320, 2], [321, 2], [322, 2], [323, 2], [324, 3], [325, 2], [326, 2], [327, 3]]


### The ILP

In [16]:
# Solve the ILP for ALL courses, setting the Cohort Limit at 110

solver = pywraplp.Solver('SGS', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

start_time = time.time()

Students = range(n)

Courses = ActualCourses
Triplets = range(len(TripletList))
LearningGroups = [1,2,3]
Blocks = ['A','B','C','D','E','F','G','H']

TripletIDs = [ [] for r in range(m)]
TripletBlocks = [ [] for r in range(8)]

for t in range(len(TripletList)):
    ID = TripletList[t][0]
    Block = TripletList[t][2]
    TripletIDs[ID].append(t)
    for r in range(8):
        if Blocks[r]==Block:
            TripletBlocks[r].append(t)
            

# define boolean variables
x = {}
for t in Triplets:
    for l in LearningGroups:
        x[t,l] = solver.IntVar(0,1, 'x[%d,%d]' % (t,l))

y = {}
for i in Students:
    for l in LearningGroups:
        y[i,l] = solver.IntVar(0,1, 'y[%d,%d]' % (i,l))

z = {}
for i in Students:
    for t in Triplets:
        for l in LearningGroups:
            z[i,t,l] = solver.IntVar(0,1, 'z[%d,%d,%d]' % (i,t,l))


            
# CONSTRAINT 0: Lock in some x[t,l] and y[i,k] variables based on previous step. Optimize all other variables.
for a in range(len(XSet)):
    t = XSet[a][0]
    l = XSet[a][1]
    solver.Add(x[t,l] == 1)
        
for b in range(len(YSet)):
    i = YSet[b][0]
    l = YSet[b][1]
    solver.Add(y[i,l] == 1)
    
        
# CONSTRAINT 1: Ensure each triplet is assigned to at most ONE learning group, unless otherwise specified
for t in Triplets:
    if t in CombinedLG:
        solver.Add(sum(x[t,l] for l in LearningGroups) <= 3)
    else:
        solver.Add(sum(x[t,l] for l in LearningGroups) == 1)


# CONSTRAINT 2: Ensure each student is assigned to at most ONE learning group
for i in Students:
    solver.Add(sum(y[i,l] for l in LearningGroups) <= 1)   


# CONSTRAINT 3: No student can take a course unless it is available to students in their learning group
for i in Students:
    for t in Triplets:
        for l in LearningGroups:
            solver.Add(z[i,t,l] <= y[i,l])
            solver.Add(z[i,t,l] <= x[t,l])

            
# CONSTRAINT 4: No student can take more than one course in a block
for i in Students:
    for r in range(8):
        solver.Add(sum(z[i,t,l] for l in LearningGroups for t in TripletBlocks[r]) <= 1)


# CONSTRAINT 5: NO student can take the same course twice       
for i in Students:
    for j in Courses:
        solver.Add(sum(z[i,t,l] for l in LearningGroups for t in TripletIDs[j]) <= 1)

            
# CONSTRAINT 6: Assign students only the course they requested(i.e. don't assign course j to a student i if P[i,j]=0)      
for i in Students:
    for j in Courses:
        if P[i,j]==0:
            solver.Add(sum(z[i,t,l] for l in LearningGroups for t in TripletIDs[j]) == 0)


# CONSTRAINT 7: No course section can exceed its prescribed capacity
for t in Triplets:
    cap = TripletCap[t]
    solver.Add(sum(z[i,t,l] for l in LearningGroups for i in Students) <= cap)

    
# CONSTRAINT 8: For the 807 must courses, the student must be assigned to a section of
# that course, though the block/section is NOT fixed.
for i in Students:
    for j in Courses:
        if P[i,j]==10:
            solver.Add(sum(z[i,t,l] for l in LearningGroups for t in TripletIDs[j]) == 1)
  
    
# CONSTRAINT 9: Assign at most 110 students to each learning group       
for l in LearningGroups:
    solver.Add(sum(y[i,l] for i in Students) <= 110) 

    
# CONSTRAINT 10: Boarding students must belong to Learning Group 1 or 2
for i in Students:
    if i in BoardingStudents:
        solver.Add(y[i,3] == 0)

        
solver.Maximize(solver.Sum(P[i,TripletList[t][0]]*z[i,t,l] 
                           for i in Students for t in Triplets for l in LearningGroups))
               
sol = solver.Solve()

print("")
print("PART 2 - Results for all courses only, with Cohort Limit of 110")
print('Optimization Complete with Total Happiness Score of', round(solver.Objective().Value()))

# compute runtime
solving_time = time.time() - start_time

print('The code ran in', round(solving_time,1), 'seconds')


PART 2 - Results for all courses only, with Cohort Limit of 110
Optimization Complete with Total Happiness Score of 11596
The code ran in 19.8 seconds


### Statistics

In [17]:
# Descriptive Statistics of our results

MustCount=0
HighCount=0
MedCount=0
LowCount=0
MustTotal=0
HighTotal=0
MedTotal=0
LowTotal=0

for i in Students:
    for j in Courses:
        if P[i,j]>0 and not j in [15,16]:
            if j in MustPriority: MustTotal +=1
            elif j in HighPriority: HighTotal +=1
            elif j in MedPriority: MedTotal +=1
            else: LowTotal +=1
        
for i in Students:
    for t in Triplets:
        for l in LearningGroups:
            if z[i,t,l].solution_value() == 1:
                j = TripletList[t][0]
                if not j in [15,16]:
                    if j in MustPriority: MustCount +=1
                    elif j in HighPriority: HighCount +=1
                    elif j in MedPriority: MedCount +=1
                    else: LowCount +=1

print(MustCount, "out of", MustTotal, "mandatory priority course assignments were made:", 
     round(100*MustCount/MustTotal, 2), "percent")
print(HighCount, "out of", HighTotal, "high priority course assignments were made:", 
     round(100*HighCount/HighTotal, 2), "percent")
print(MedCount, "out of", MedTotal, "medium priority course assignments were made:", 
     round(100*MedCount/MedTotal, 2), "percent")
print(LowCount, "out of", LowTotal, "low priority course assignments were made:", 
     round(100*LowCount/LowTotal, 2), "percent")

print("")
for l in LearningGroups:
    count=0
    for i in Students:
        if y[i,l].solution_value()==1:
            count+=1
    print("Learning Group", l, "has", count, "students")

807 out of 807 mandatory priority course assignments were made: 100.0 percent
514 out of 541 high priority course assignments were made: 95.01 percent
144 out of 166 medium priority course assignments were made: 86.75 percent
524 out of 789 low priority course assignments were made: 66.41 percent

Learning Group 1 has 110 students
Learning Group 2 has 108 students
Learning Group 3 has 110 students


### The Timetable

In [18]:
# Generate three .txt files containing the 
# three sets of variables x[t,l], y[i,l], z[i,t,l]
# optimized by our program

XSet=[]
    
for t in Triplets:
    for l in LearningGroups:
        if x[t,l].solution_value()==1:
            XSet.append([t,l])
            
file = open('XSet.txt', 'w')
file.write(str(XSet))
file.close()

YSet=[]
    
for i in Students:
    for l in LearningGroups:
        if y[i,l].solution_value()==1:
            YSet.append([i,l])
            
file = open('YSet.txt', 'w')
file.write(str(YSet))
file.close()

ZSet=[]
for i in Students:
    for t in Triplets:
        for l in LearningGroups:
            if z[i,t,l].solution_value()==1:
                ZSet.append([i,t,l])
            
file = open('ZSet.txt', 'w')
file.write(str(ZSet))
file.close()

TotRequests = MustTotal+HighTotal+MedTotal+LowTotal
print('Our program satisfied a total of', len(ZSet), 'course assignments out of', 
      TotRequests, 'total course requests, i.e.', round(len(ZSet)/TotRequests,2))

Our program satisfied a total of 1989 course assignments out of 2303 total course requests, i.e. 0.86
